In [1]:
import requests
headers = {'User-Agent': 'Rishabh K Sharma/1.0 (Contact: sharmarishabhk@gmail.com)'}
response = requests.get('https://example.com', headers=headers)

response

<Response [200]>

In [2]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun


In [4]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper)


In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [15]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))

retriever = vectordb.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e42fa8183b0>, search_kwargs={})

In [16]:
from langchain_classic.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, 'langsmith_search', 'Search for information about langsmith. for any question about langsmith use this tool')

In [17]:
retriever_tool.name

'langsmith_search'

In [18]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper = arxiv_wrapper)

arxiv.name

'arxiv'

In [19]:
tools = [wiki, arxiv, retriever_tool] 

In [20]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/rishabh/PycharmProjects/LangChain/.venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about langsmith. for any question about langsmith use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7e42fae789a0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_communi

In [21]:
from langchain_ollama.chat_models import ChatOllama

llm = ChatOllama(model="Llama3.2:3B", temprature=0)

In [22]:
llm

ChatOllama(model='Llama3.2:3B')

In [23]:
from langchain_classic import hub

prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
from langchain_classic.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)


In [25]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [26]:
from langchain_classic.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [27]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [30]:
agent_executor.invoke({"input": "Dr. Sanjeev D Choube"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Dr. Sanjeev D Choube'}`


Edit the source of this page on GitHub.
Connect these docs programmatically to Claude, VSCode, and more via MCP for    real-time answers.Was this page helpful?YesNoPricing plansNextDocs by LangChain home pagegithubxlinkedinyoutubeResourcesForumChangelogLangChain AcademyTrust CenterCompanyAboutCareersBloggithubxlinkedinyoutubePowered by Mintlify

LangSmith meets the highest standards of data security and privacy with HIPAA, SOC 2 Type 2, and GDPR compliance. For more information, see the Trust Center.
​Workflow
LangSmith combines observability, evaluation, deployment, and platform setup in one integrated workflow—from local development to production.

LangSmith docs - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSear

{'input': 'Dr. Sanjeev D Choube',
 'output': "It appears that Dr. Sanjeev D Choube is not a public figure with widely available information. Can you please provide more context or clarify who Dr. Sanjeev D Choube is? I'll do my best to find relevant information and provide an answer to your original question.\n\nIf you could provide more details about what you're looking for, such as their field of work or any specific projects they might be involved in, I can try to help you better."}

In [32]:
from IPython.display import display, Markdown

data = {'input': 'Mujhe sim leni hai par mujhe pata nahi hai mai kya karu',
 'output': 'मुझे खेद है, लेकिन मैंने आपको यह स्पष्ट करना चाहूंगा कि आप एक विशिष्ट प्रश्न पूछ रहे थे, जिसका उत्तर देने में मैं सक्षम नहीं हुआ। लेकिन मैं आपकी सहायता के लिए यहाँ हूँ।\n\nआपके पूछे गए प्रश्न का सारांश यह है: "मुझे सिम लेनी है, लेकिन मुझे पता नहीं है कि मैं क्या करूँ।"\n\nयहाँ कुछ सुझाव दिए गए हैं जो आपको अपने सिम लेने में मदद कर सकते हैं:\n\n1. **वाईडफोन आईडिया वेबसाइट पर जाएं**: आप वाईडफोन आईडिया की आधिकारिक वेबसाइट पर जाकर अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं। यहाँ आपको विभिन्न प्लान और पैक के विकल्प मिलेंगे।\n2. **स्टोर में जाएं**: आप वाईडफोन आईडिया के निकटतम स्टोर पर जाकर अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं। यहाँ आपको सहायता प्रदान करने वाले कर्मचारी मिलेंगे।\n3. **ऑटो-रिचर्ज टेलीफोन पर कॉल करें**: आप वाईडफोन आईडिया के ऑटो-रिचर्ज टेलीफोन पर कॉल करके अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं।\n4. **सोशल मीडिया पर जानकारी प्राप्त करें**: आप वाईडफोन आईडिया के आधिकारिक सोशल मीडिया अकाउंट्स पर जाकर अपने सिम लेने की प्रक्रिया के बारे में जानकारी प्राप्त कर सकते हैं।\n\nमुझे आशा है कि ये सुझाव आपको अपने सिम लेने में मदद करेंगे। यदि आपको और अधिक सहायता की आवश्यकता है, तो कृपया मुझसे पूछें।'}

markdown_text = f"""
### User Input

{data['input']}

### System Output

{data['output']}
"""

display(Markdown(markdown_text))



### User Input

Mujhe sim leni hai par mujhe pata nahi hai mai kya karu

### System Output

मुझे खेद है, लेकिन मैंने आपको यह स्पष्ट करना चाहूंगा कि आप एक विशिष्ट प्रश्न पूछ रहे थे, जिसका उत्तर देने में मैं सक्षम नहीं हुआ। लेकिन मैं आपकी सहायता के लिए यहाँ हूँ।

आपके पूछे गए प्रश्न का सारांश यह है: "मुझे सिम लेनी है, लेकिन मुझे पता नहीं है कि मैं क्या करूँ।"

यहाँ कुछ सुझाव दिए गए हैं जो आपको अपने सिम लेने में मदद कर सकते हैं:

1. **वाईडफोन आईडिया वेबसाइट पर जाएं**: आप वाईडफोन आईडिया की आधिकारिक वेबसाइट पर जाकर अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं। यहाँ आपको विभिन्न प्लान और पैक के विकल्प मिलेंगे।
2. **स्टोर में जाएं**: आप वाईडफोन आईडिया के निकटतम स्टोर पर जाकर अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं। यहाँ आपको सहायता प्रदान करने वाले कर्मचारी मिलेंगे।
3. **ऑटो-रिचर्ज टेलीफोन पर कॉल करें**: आप वाईडफोन आईडिया के ऑटो-रिचर्ज टेलीफोन पर कॉल करके अपने सिम लेने की प्रक्रिया शुरू कर सकते हैं।
4. **सोशल मीडिया पर जानकारी प्राप्त करें**: आप वाईडफोन आईडिया के आधिकारिक सोशल मीडिया अकाउंट्स पर जाकर अपने सिम लेने की प्रक्रिया के बारे में जानकारी प्राप्त कर सकते हैं।

मुझे आशा है कि ये सुझाव आपको अपने सिम लेने में मदद करेंगे। यदि आपको और अधिक सहायता की आवश्यकता है, तो कृपया मुझसे पूछें।
